In [33]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt

In [34]:
df = pd.read_csv('dataset.csv')
df.head(3)

,Unnamed: 0,Job Title,Job Description
0,0,Flutter Developer,We are looking for hire experts flutter develo...
1,1,Django Developer,PYTHON/DJANGO (Developer/Lead) - Job Code(PDJ ...
2,2,Machine Learning,"Data Scientist (Contractor)\n\nBangalore, IN\n..."


In [35]:
df.shape

(2277, 3)

## Data Cleaning

In [36]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2277 entries, 0 to 2276
Data columns (total 3 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Unnamed: 0       2277 non-null   int64 
 1   Job Title        2277 non-null   object
 2   Job Description  2277 non-null   object
dtypes: int64(1), object(2)
memory usage: 53.5+ KB


In [37]:
df.drop(columns=['Unnamed: 0'],inplace=True)

In [38]:
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()
df['Job Title'] = encoder.fit_transform(df['Job Title'])

In [39]:
df.head(3)

,Job Title,Job Description
0,4,We are looking for hire experts flutter develo...
1,3,PYTHON/DJANGO (Developer/Lead) - Job Code(PDJ ...
2,8,"Data Scientist (Contractor)\n\nBangalore, IN\n..."


In [40]:
df.isnull().sum()

Job Title          0
Job Description    0
dtype: int64

In [41]:
df.duplicated().sum()

np.int64(0)

In [42]:
import nltk
nltk.download('punkt')
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\vikas\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\vikas\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

## Data Preprocessing

In [43]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\vikas\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [44]:
# import nltk
# from nltk.corpus import stopwords
# import string
# from nltk.stem.porter import PorterStemmer
# ps = PorterStemmer()

# def transform_text(text):
#     text = text.lower()
#     text = nltk.word_tokenize(text)
#     y = []
#     for i in text:
#         if i.isalnum():   
#             y.append(i)
#     text = y[:]   
#     y.clear()     
    
#     for i in text:
#         if i not in stopwords.words('english') and i not in string.punctuation:
#             y.append(i)
#     text = y[:]
#     y.clear()

#     for i in text:
#         y.append(ps.stem(i))
    
#     return " ".join(y)

import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()

nltk.download('stopwords')
nltk.download('wordnet')

stop_words = set(stopwords.words('english'))
lm = WordNetLemmatizer()
result = []
def transform_text(text):
    # Lowercase + keep only words
    tokens = re.findall(r'\b[a-zA-Z]+\b', text.lower())
    
    # Remove stopwords
    tokens = [w for w in tokens if w not in stop_words]
    
    # Lemmatization
    tokens = [lm.lemmatize(w) for w in tokens]
    
    for i in tokens:
        result.append(ps.stem(i))

        

    return " ".join(result)


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\vikas\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\vikas\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [45]:
df['transformed_text']=df['Job Description'].apply(transform_text)

In [46]:
df.head(3)

,Job Title,Job Description,transformed_text
0,4,We are looking for hire experts flutter develo...,look hire expert flutter develop elig post app...
1,3,PYTHON/DJANGO (Developer/Lead) - Job Code(PDJ ...,look hire expert flutter develop elig post app...
2,8,"Data Scientist (Contractor)\n\nBangalore, IN\n...",look hire expert flutter develop elig post app...


## Model Building

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(
    max_features=1000, 
    tokenizer=lambda x: x.split()   
)
X = vectorizer.fit_transform(df['transformed_text'])


c:\Users\vikas\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\feature_extraction\text.py:517: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


In [ ]:
from sklearn.cluster import KMeans


wcss = []
for i in range(2, 21):
    km = KMeans(n_clusters=i, init='k-means++', random_state=42, n_init=10)
    km.fit(X)
    wcss.append(km.inertia_)

plt.figure(figsize=(10, 5))
plt.plot(range(2, 21), wcss, marker='o', linestyle='-', color='b')
plt.title("Elbow Method for Optimal k", fontsize=14)
plt.xlabel("Number of Clusters (k)", fontsize=12)
plt.ylabel("WCSS (Within-Cluster Sum of Squares)", fontsize=12)
plt.xticks(range(2, 21))
plt.grid(True, linestyle="--", alpha=0.6)
plt.show()


ValueError: Expected a 2-dimensional container but got <class 'pandas.core.series.Series'> instead. Pass a DataFrame containing a single row (i.e. single sample) or a single column (i.e. single feature) instead.

In [ ]:
best_k = 5 
kmn = KMeans(n_clusters=best_k, init='k-means++', random_state=42, n_init=10)
df['predict_Output'] = kmn.fit_predict(X)
